In [ ]:
pip install openpyxl

In [ ]:
import os
import pandas as pd

# 제어 문자를 제거하는 함수
def remove_control_characters(s):
    return "".join(ch for ch in s if ch.isprintable())

# 데이터 및 설정
loaded_config = {
    'general': {
        'data_path': '/data/ephemeral/home/data/data'  # 데이터 경로 설정
    }
}
data_path = loaded_config['general']['data_path']

# CSV 파일 불러오기
train_df = pd.read_csv(os.path.join(data_path, 'train.csv'))

# 'dialogue' 열에 제어 문자 제거 함수 적용
train_df['dialogue'] = train_df['dialogue'].apply(remove_control_characters)

# 확인을 위해 처음 몇 줄 출력
print(train_df.head())

# 필요한 경우 수정된 데이터프레임 저장
train_df.to_csv(os.path.join(data_path, 'train_clean.csv'), index=False)


In [ ]:
import os
import pandas as pd
from openai import OpenAI

# API 설정
client = OpenAI(
    api_key="up_kts6Q2KQWy94negmKwvQcos7cYo6O",  # 실제 API 키 입력
    base_url="https://api.upstage.ai/v1/solar"
)

def translate_text(text, model="solar-1-mini-translate-koen"):
    """ 주어진 텍스트를 영어로 번역합니다. """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": text}
        ],
        stream=False
    )
    return response.choices[0].message.content if response.choices else text

# 데이터 및 설정
loaded_config = {
    'general': {
        'data_path': '/data/ephemeral/home/data/data'  # 데이터 경로 설정
    }
}
data_path = loaded_config['general']['data_path']
train_df = pd.read_csv(os.path.join(data_path, 'train_clean.csv'))
batch_size = 100
columns_to_translate = ['dialogue', 'summary', 'topic']

# 배치 3부터 시작하기 위한 계산 (배치 인덱스는 0부터 시작하므로 2 * batch_size)
restart_batch = 65  # 0-indexed, 배치 3은 인덱스 2입니다.
start_index = restart_batch * batch_size

# 배치 처리 및 Excel 파일 저장
for start in range(start_index, train_df.shape[0], batch_size):
    end = min(start + batch_size, train_df.shape[0])
    batch = train_df.iloc[start:end]
    for column in columns_to_translate:
        batch[column + '_en'] = batch[column].apply(translate_text)
    batch_file = os.path.join(data_path, f'batch_{start // batch_size + 1}.xlsx')
    batch.to_excel(batch_file, index=False)
    print(f"Processed and saved batch {start // batch_size + 1}")

# 모든 배치 파일 병합
all_files = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.startswith('batch_') and f.endswith('.xlsx')]
all_data = pd.concat((pd.read_excel(f) for f in all_files), ignore_index=True)

# 최종 파일 저장
output_file = os.path.join(data_path, 'train_en.csv')
all_data.to_csv(output_file, index=False)
print("All data merged and saved to", output_file)

#18.68시간 예상
